In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import yaml

In [109]:
blue_shades = [
    "#1f4fd8",
    "#5779c9",
    "#203866",
    "#0055ff",
]
green_shades = [
    "#06d945",
    "#1da81d",
    "#1fc93e",
    "#1EA619",
]
orange_shades = [
    "#ff7f0e",
    "#ff9f3f",
    "#ff6f0e",
    "#ff5a00",
]
red_shade = "#ff0044"

vis_dict = {
    'M1': {'color':red_shade},
    'M2': {'color':green_shades[0]},
    'M3': {'color':green_shades[1]},
    'M4': {'color':green_shades[2]},
    'M5': {'color':green_shades[3]},
    'M6': {'color':blue_shades[0]},
    'M7': {'color':blue_shades[1]},
    'M8': {'color':blue_shades[2]},
    'M9': {'color':blue_shades[3]},
    'M10': {'color':orange_shades[0]},
    'M11': {'color':orange_shades[1]},
    'M12': {'color':orange_shades[2]},    
}

In [98]:
dir = '/Volumes/DL_Share2/Boris/Devices/AFM Press [P]/MS2P/MS2P_5/05 Data'

runs_file = 'MS2P5_runs.yml'
runs_data = yaml.safe_load(open(os.path.join(dir, runs_file)))
run_ids = ['MS2P5_run03']

run_id = run_ids[0]
run_dict = runs_data.get(run_id)

data_dir = os.path.join(dir, 'raw')
filenames = [f for f in os.listdir(data_dir) if run_id in f]

In [99]:
filenames

['MS2P5_run03_M8.txt', 'MS2P5_run03_M1.txt', 'MS2P5_run03_M6.txt']

In [100]:
run_dict['acquisitions']['drain']

['M6', 'M8', 'M1']

In [101]:
dataframe_parameters = run_dict.get('dataframe_parameters', {})
delimeter = dataframe_parameters.get('delimeter') 
columns = dataframe_parameters.get('data_columns', [])  
device = run_dict.get('device')
device

'MS2P5'

In [102]:
df = pd.read_csv(os.path.join(data_dir, filenames[0]), sep=delimeter)
df = df[df.columns[:-1]]
df.columns = columns
df

/var/folders/5h/80_mph8s1tg289gn7wkq8qq00000gn/T/ipykernel_81525/216102649.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.



,Time,Idc,Vdc,Iac_x,Iac_y
0,25.5120,5.995100e-07,2.9561,-5.270900e-10,9.333200e-11
1,27.3889,5.846600e-07,2.9293,-4.968300e-10,1.069500e-10
2,28.8258,5.713500e-07,2.9053,-4.923500e-10,1.069700e-10
3,30.2600,5.582300e-07,2.8815,-4.701400e-10,1.777500e-10
4,31.6847,5.455400e-07,2.8577,-4.811900e-10,1.244000e-10
...,...,...,...,...,...
496,744.9898,5.781300e-07,2.9227,-4.846400e-10,8.683600e-11
497,746.4161,5.907900e-07,2.9464,-5.177700e-10,1.212500e-10
498,747.8334,6.038300e-07,2.9701,-5.109200e-10,1.119800e-10
499,749.2528,6.170000e-07,2.9938,-5.035500e-10,9.942000e-11


In [103]:
def account_for_resistor(df, R_series=0.991e6): #in ohms
    df['Vdc_crct'] = df['Vdc'] - df['Idc'] * R_series
    return df

In [104]:
def build_dfs(directory):
    files = [f for f in os.listdir(data_dir) if run_id in f]
    dfs= {}
    labels = run_dict['acquisitions']['drain']

    for label in labels:
        file = [f for f in files if label in f][0]
        df = pd.read_csv(os.path.join(directory, file), sep=delimeter)
        df = df[df.columns[:-1]]
        df.columns = columns
        df = account_for_resistor(df)
        dfs[label] = df

    labels = sorted(labels, key=lambda x: int(x[1:]))

    return dfs, labels

In [114]:
def plotter(fig, dfs, labels, device):
    for label in labels[::-1]:
        df = dfs[label]

        # LEFT y-axis
        fig.add_trace(
            go.Scatter(
                x=df['Vdc_crct'],
                y=df['Idc'],
                mode='lines',
                name=f'{label}',
                line=dict(width=2, color=vis_dict[label]['color']),
                yaxis='y'
            )
        )

        # RIGHT y-axis
        fig.add_trace(
            go.Scatter(
                x=df['Vdc_crct'],
                y=-df['Iac_x'],
                mode='lines',
                name=f'{label}_Iac_x',
                line=dict(width=2, color=vis_dict[label]['color'], dash='dash'),
                yaxis='y2'
            )
        )

    fig.update_layout(
        xaxis=dict(title='V, V'),

        yaxis=dict(
            title='Idc, A',
        ),

        yaxis2=dict(
            title='Iac_x, A',
            overlaying='y',
            side='right'
        ),

        title=f"{device} corrected",
        width=800,
        height=500
    )

    fig.show()


In [115]:
fig = go.Figure()
dfs, labels = build_dfs(data_dir)
plotter(fig, dfs, labels, device)

/var/folders/5h/80_mph8s1tg289gn7wkq8qq00000gn/T/ipykernel_81525/1404465221.py:8: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.

/var/folders/5h/80_mph8s1tg289gn7wkq8qq00000gn/T/ipykernel_81525/1404465221.py:8: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.

/var/folders/5h/80_mph8s1tg289gn7wkq8qq00000gn/T/ipykernel_81525/1404465221.py:8: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.

